<a href="https://colab.research.google.com/github/sominshim/Predicting_Personality_through_Text/blob/main/mbti_crawling_ver_KOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (110 kB/s)

In [64]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from pandas import Series, DataFrame
import pandas as pd
import time
import re
from tqdm import tqdm_notebook

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


In [65]:
USER = 'wkdthals325'
PASS = 'Shim5186!'

wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

#로그인 페이지 접근

url_login = "https://nid.naver.com/nidlogin.login"

wd.get(url_login)

#텍스트박스에 아이디, 비밀번호 입력하기

e = wd.find_element_by_id("id")

e.clear()

e.send_keys(USER)

e = wd.find_element_by_id("pw")

e.clear()

e.send_keys(PASS)


#입력 양식 전송, 로그인

form = wd.find_element_by_css_selector("input.btn_global[type=submit]")

form.submit()

#print("네이버에 로그인...")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [66]:
def crawler(url_sub):
  contents = []
  names = []
  for link in tqdm_notebook(url_sub):
    try:
      # print(link)
      wd.get(link)
      time.sleep(1)
      wd.switch_to.frame('cafe_main')
      # time.sleep(1)
      soup = BeautifulSoup(wd.page_source, 'lxml')
      contentArea = soup.find('div', {'class':'article_viewer'})
      parags = contentArea.findAll('p')  
      content = ""
      # print(parags)
      for parag in parags:
          content += parag.text.strip()
      contents.append(content)

      nameArea = soup.find('div', {'class':'nick_box'})
      name_parags = nameArea.findAll('a')  
      name = ""
      # print(parags)
      for parag in name_parags:
          name += parag.text.strip()
      name = re.sub('[^a-zA-Z]','',name.upper())
      names.append(name)
      pass
    except:
      # Continue to next iteration.
      continue

  return contents, names

In [68]:
iter = 3
for i in range(iter):
  # url = mbti_url + str(i+1)
  wd.get('https://cafe.naver.com/ArticleList.nhn?search.clubid=11856775&search.boardtype=L&search.totalCount=151&search.page='+ str(i+1))  #mbti카페
  time.sleep(1)
  wd.switch_to.frame('cafe_main')  #카페 메인 아이프레임
  html = wd.page_source  #페이지 소스 가져오기
  #print(html)
  # wd.close()

  soup = BeautifulSoup(html, "html.parser")

  title = soup.select('.inner_list')

  url_sub = [] # 각 게시물 url 뽑음
  for link in title:
    # print(link)
    linkadd = link.find("a")['href']
    url_sub.append("https://cafe.naver.com" + linkadd)
    
  df1, df2 = crawler(url_sub)
  df1 = pd.DataFrame(data = df1, columns=['posts'])
  df2 = pd.DataFrame(data = df2, columns=['type'])
  add = pd.concat([df2, df1], axis=1)

  if i == 0:
    df = add
  else:
    df = pd.concat([df, add], axis=0)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [69]:
df

,type,posts
0,,불안한 것이 당연합니다- 어른을 위한 단단한 마음 수업 -한덕현 저 | 한빛비즈​...
1,,안녕하세요? 카가입니다.​오랜만에 프로그램에 참여하고 후기를 남겨봅니다.제가 참여한...
2,EG,"마음의 질문 하나가,당신의 내면에 변화와 성장으로 이끕니다.나를 일깨우다. 여강현...."
3,,​<심리자격증 28종 무료수강지원 전액지원 이벤트 신청방법>​​1. 홈페이지 회원가...
4,,안녕하세요?^^ 엠헬 수석부매니저 카가 성기원입니다.근래에 제게 MBTI Form ...
5,,"안녕하세요? MBTI&HEALTH 수석부매니저 카가입니다.예전부터 온라인, 오프라인..."
6,,안녕하세요? MBTI&HEATLH 카페에 오신것을 환영합니다.이하는 MBTI&HEA...
7,EG,모임은 혼자가 아니라 여러 사람이 함께 참여하는 자리입니다. 모임 참여자는 모임의 ...
8,,"스탭(정예 스탭, 명예 스탭, 강사 스탭) 이외에 일반 회원의 정모 및 벙개, 단톡..."
9,ESTP,1. 최근 카페의 질의응답 게시판에 지속적으로 등업을 요구하는 글이 올라오고 있습니...
